Calculating absorption spectra

In this exercise we are going to calculate the absorption spectra of the material you have considered so far.
In general, the absorption spectrum is given by the imaginary part of the macroscopic dielectric function. This means that our computational taks is to calculate the macroscopic dielectric funciton for our material, and then plot the imaginary part. For more information about the dielectric function please consult:

https://wiki.fysik.dtu.dk/gpaw/tutorials/dielectric_response/dielectric_response.html#df-tutorial

To calculate the dielectric function, we will for this case use the Random Phase Approximation (RPA) correlation energy (so we say we calculate the absorption spectrum within the random phase approximation). Details about RPA can be foun here:

https://wiki.fysik.dtu.dk/gpaw/documentation/xc/rpa.html#rpa


As discussed earlier, it is of greatest importance to make sure our calculations are converged. Since the convergence parameters is not necessarily the same for band gaps and RPA absorption spectra (and any other material property) with respect to for instance plane-wave cut-off or k-points, we need to do do a new groundstate calculation with parameters. For RPA absorption spectra we will here look at the number of bands included in the calculation and the k-point mesh. We will therefore restart from the previous groundstate file, and update some of the parameters. First we will look at a too rough k-point mesh and then do more calculations with a finer k-point mesh. In this way we can see the importance of converging the calculations.

The new groundstate is calculated below. 

In [ ]:
import os
import numpy as np
import gpaw as GPAW
from gpaw.mpi import world
from gpaw.occupations import FermiDirac
from gpaw.response.df import DielectricFunction
import matplotlib.pyplot as plt


#Load ground state gpw file
calc_old = GPAW('gs.gpw', txt=None)

#Extract number of valence bands:
nval = calc_old.wfs.nvalence

#Do new ground state calculations with more k-points.
#This is because in general RPA calculations requires more k-poins to be converged.

calc = GPAW('gs.gpw', 
             fixdensity=True,				#keep electron density from ground state calculation fixed
             kpts=(24,24,1),				#Number of k-points for BN
             nbands = 4*nval,				#Number of bands to include in calculation
             convergence={'bands': 3*nval},		#Number of bands to convergence
             txt='es.txt',
             occupations=FermiDirac(width=1e-4))


calc.get_potential_energy()
calc.write('BN_24x24x1.gpw', 'all')			#'all' means we save all the wave functions to the gpw file. This is required for the rpa calculations

Now that we have obtained the groundstate, it is time to calculate the dielectric function. This is done below where we first initialize the parameters needed for an RPA calculation, then calculate the dielectric funcition, and finally obtain the polarizability. Note that one also would have to converge the parameters initialized below, for a fully converged study, however in the interest of time and computational power we will not consider this here. The principle is however exactly the same, as you will see with the different k-point meshes we will emply here.

In [ ]:
#Define parameters for rpa calculations:
kwargs = {'truncation': '2D',				#Truncate the Coulomb interaction in the lateral direction to avoid non-physical interactions due to periodic boundary conditions
          'eta': 0.05,					#Broadening parameter
          'domega0': 0.01,				#Define spacing of frequency grid
          'intraband': False,				#In this task we do not include intraband transitions for calculating the absorption spectrum
          'nblocks': 8,					#Number of blocks used for parallization
          'ecut': 50,					#Plane wave cutoff in eV
          'nbands': 24,					#Number of bands included in rpa calculation
          'name': 'chi_BN_24x24x1'}			#Name of density response function .pckl file


#Calculate dielectric function. Takes groundstate calculation and defined parameters in "kwargs" as input:
df = DielectricFunction('BN_24x24x1.gpw', **kwargs)


#Finally we calculate he polarizability in the x, y, and z direction. The output is a .csv file (one for each direction) which can be plotted.
df.get_polarizability(xc='RPA',				#We want to calculate the absorption spectrum within RPA
                      q_c = [0, 0, 0],			#We consider the zero momentum wave vector
                      pbc = [True, True, False],	#Define boundary conditions
                      direction = 'x', 			#Define real space direction
                      filename='BN_24x24x1_rpa_x.csv')	#Name of output file

df.get_polarizability(xc='RPA',
                      q_c = [0, 0, 0],
                      pbc = [True, True, False],
                      direction = 'y', 
                      filename='BN_24x24x1_rpa_y.csv')

df.get_polarizability(xc='RPA',
                      q_c = [0, 0, 0],
                      pbc = [True, True, False],
                      direction = 'z', 
                      filename='BN_24x24x1_rpa_z.csv')

Now it is time to do more calculations with finer k-point mesh to see the importance of converging our calculations. Repeat the above calculations with the following paramters:

For BN try: (24,24,1), (40,40,1), and (60,60,1) k-points
For bulk materials try: (12,12,4), (18,18,6), and (24,24,8) k-points

